Rohit BHALERAO Individual Assignment

# Data Cleaning

In [1]:
#import packages

import pandas as pd
import numpy as np
pd.options.mode.chained_assignment = None

In [2]:
#Read raw file scraped from website

data_df = pd.read_csv('raw_data.csv')
data_df

,category,address,price
0,Maison 4 pièces 93 m²\nLille 59000 (Lille-Sud),Lille 59000 (Lille-Sud),192 000 €
1,Maison 4 pièces 117 m²\nLille 59800 (Saint-Mau...,Lille 59800 (Saint-Maurice - Pellevoisin),325 500 €
2,VILL'ARBOREA (appartements neufs 1 à 5 pièces)...,Lille 59160 (Bourg - Délivrance),170 000 à 385 000 €
3,LILLE VILLA ULMA (appartements neufs 1 à 3 piè...,Lomme 59160,176 400 à 316 320 €
4,EDENIUM (appartements neufs 2 à 5 pièces)\nLil...,Lille 59000 (Lille-Sud),111 051 à 317 000 €
...,...,...,...
604,Appartement neuf 5 pièces\nLille 59800 (Fives),Lille 59800 (Fives),171 000 à 384 500 €
605,Programme Immobilier à LILLE (appartements neu...,Lille 59000 (Bourg - Délivrance),257 250 €
606,Programme Immobilier à LILLE (appartements neu...,Lille 59000 (Wazemmes),222 750 à 434 775 €
607,Appartement neuf 2 pièces 42 m²\nLille 59800 (...,Lille 59800 (Centre-ville - Euralille),NaN


In [3]:
#Remove rows without price data
data_df = data_df.dropna(axis=0, subset=['price'])

In [4]:
#Adding column for negotiable or non-negotiable property prices
data_df['Negotiable'] = data_df['price'].str.contains('à')
data_df["Negotiable"] = data_df.loc[:4].replace({False: "No", True: "Yes"}, inplace=True)
data_df.head(3)

,category,address,price,Negotiable
0,Maison 4 pièces 93 m²\nLille 59000 (Lille-Sud),Lille 59000 (Lille-Sud),192 000 €,None
1,Maison 4 pièces 117 m²\nLille 59800 (Saint-Mau...,Lille 59800 (Saint-Maurice - Pellevoisin),325 500 €,None
2,VILL'ARBOREA (appartements neufs 1 à 5 pièces)...,Lille 59160 (Bourg - Délivrance),170 000 à 385 000 €,None


In [5]:
#Converting price column from string to integer
data_df['price'] = data_df['price'].str.replace(' ','')
data_df['price'] = data_df['price'].str.replace('à','+')
data_df['price'] = data_df['price'].str.replace('€','')

data_df = data_df[~data_df.price.str.contains("partir", na = False)] #removing 2 rows with text for prices

data_df['price'] = data_df['price'].map(lambda price: eval(price))
data_df['price'] = np.where((data_df.Negotiable == 'Yes'),data_df['price']/2,data_df.price) 
#calculate mean price for negotiable prices (original price was a range, e.g., 200,000£ to 230,000£)
data_df.head(3)


,category,address,price,Negotiable
0,Maison 4 pièces 93 m²\nLille 59000 (Lille-Sud),Lille 59000 (Lille-Sud),192000.0,None
1,Maison 4 pièces 117 m²\nLille 59800 (Saint-Mau...,Lille 59800 (Saint-Maurice - Pellevoisin),325500.0,None
2,VILL'ARBOREA (appartements neufs 1 à 5 pièces)...,Lille 59160 (Bourg - Délivrance),555000.0,None


In [6]:
#Adding pincode column by extracting the number from address string
data_df['pincode'] = data_df['address'].str.extract('(\d+)', expand=False)
data_df['pincode'] = data_df['pincode'].transform(lambda x: int(x))
data_df.head(3)

,category,address,price,Negotiable,pincode
0,Maison 4 pièces 93 m²\nLille 59000 (Lille-Sud),Lille 59000 (Lille-Sud),192000.0,None,59000
1,Maison 4 pièces 117 m²\nLille 59800 (Saint-Mau...,Lille 59800 (Saint-Maurice - Pellevoisin),325500.0,None,59800
2,VILL'ARBOREA (appartements neufs 1 à 5 pièces)...,Lille 59160 (Bourg - Délivrance),555000.0,None,59160


In [7]:
#Adding column locality from address by extracting sting between parenthesis () 
data_df['locality'] = data_df['address'].apply(lambda st: st[st.find("(")+1:st.find(")")+1])
data_df['locality'] = data_df['locality'].str.replace(')','')
data_df.head(3)

,category,address,price,Negotiable,pincode,locality
0,Maison 4 pièces 93 m²\nLille 59000 (Lille-Sud),Lille 59000 (Lille-Sud),192000.0,None,59000,Lille-Sud
1,Maison 4 pièces 117 m²\nLille 59800 (Saint-Mau...,Lille 59800 (Saint-Maurice - Pellevoisin),325500.0,None,59800,Saint-Maurice - Pellevoisin
2,VILL'ARBOREA (appartements neufs 1 à 5 pièces)...,Lille 59160 (Bourg - Délivrance),555000.0,None,59160,Bourg - Délivrance


In [8]:
#Adding column type of property from category (Not perfect)
data_df['Type'] = data_df['category'].transform(lambda t: t.str.split(' ',1).str[0])
data_df
data_df.head(3)

,category,address,price,Negotiable,pincode,locality,Type
0,Maison 4 pièces 93 m²\nLille 59000 (Lille-Sud),Lille 59000 (Lille-Sud),192000.0,None,59000,Lille-Sud,Maison
1,Maison 4 pièces 117 m²\nLille 59800 (Saint-Mau...,Lille 59800 (Saint-Maurice - Pellevoisin),325500.0,None,59800,Saint-Maurice - Pellevoisin,Maison
2,VILL'ARBOREA (appartements neufs 1 à 5 pièces)...,Lille 59160 (Bourg - Délivrance),555000.0,None,59160,Bourg - Délivrance,VILL'ARBOREA


In [9]:
#Adding column number of rooms as first digits from category column (Not perfect)
data_df['nbr_rooms'] = data_df['category'].str.extract('(\d+)', expand=False)
data_df['nbr_rooms'] = data_df['nbr_rooms'].transform(lambda x: float(x))
data_df.head(3)

,category,address,price,Negotiable,pincode,locality,Type,nbr_rooms
0,Maison 4 pièces 93 m²\nLille 59000 (Lille-Sud),Lille 59000 (Lille-Sud),192000.0,None,59000,Lille-Sud,Maison,4.0
1,Maison 4 pièces 117 m²\nLille 59800 (Saint-Mau...,Lille 59800 (Saint-Maurice - Pellevoisin),325500.0,None,59800,Saint-Maurice - Pellevoisin,Maison,4.0
2,VILL'ARBOREA (appartements neufs 1 à 5 pièces)...,Lille 59160 (Bourg - Délivrance),555000.0,None,59160,Bourg - Délivrance,VILL'ARBOREA,1.0


In [10]:
#Adding column area (Remove first part of the string containing the number of rooms, then extract the digits)
data_df['area'] = data_df['category'].transform(lambda x: x.str.split('pièce').str[1])
data_df['area'] = data_df['area'].str.extract('(\d+)', expand=False)
data_df['area'] = data_df['area'].transform(lambda x: float(x))
data_df['area'] = np.where(data_df['area'] < 1000,data_df['area'],np.nan)
data_df.head(3)

,category,address,price,Negotiable,pincode,locality,Type,nbr_rooms,area
0,Maison 4 pièces 93 m²\nLille 59000 (Lille-Sud),Lille 59000 (Lille-Sud),192000.0,None,59000,Lille-Sud,Maison,4.0,93.0
1,Maison 4 pièces 117 m²\nLille 59800 (Saint-Mau...,Lille 59800 (Saint-Maurice - Pellevoisin),325500.0,None,59800,Saint-Maurice - Pellevoisin,Maison,4.0,117.0
2,VILL'ARBOREA (appartements neufs 1 à 5 pièces)...,Lille 59160 (Bourg - Délivrance),555000.0,None,59160,Bourg - Délivrance,VILL'ARBOREA,1.0,NaN


In [11]:
#Read the cleaned data to a .csv file
data_df.to_csv(r'C:\Users\rbhalerao\Desktop\Financial Programming Individual Assignment Rohit BHALERAO\Clean_data1.csv', index=False) 